<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/konlp_tf.data_pract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NLP-kr/tensorflow-ml-nlp.git

fatal: destination path 'tensorflow-ml-nlp' already exists and is not an empty directory.


In [2]:
!ls

sample_data  tensorflow-ml-nlp


In [3]:
import os
os.chdir('/content/tensorflow-ml-nlp')
!ls

1.Intro     3.NLP_INTRO		   5.TEXT_SIM  main.png   requirements.txt
2.NLP_PREP  4.TEXT_CLASSIFICATION  6.CHATBOT   README.md


In [4]:
!pip install -r requirements.txt

In [0]:
# tf.data 실습
import os 
import tensorflow as tf
import numpy as np

from tensorflow.keras import preprocessing

In [0]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

label = [[1],[0],[1],[1],[0],[1]]

In [0]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index

In [8]:
print("토큰화 된 배열: ", sequences)
print("각 단어의 인덱스: {}", word_index)
print("라벨: ",label)

토큰화 된 배열:  [[4, 1, 5, 6], [7, 1, 8, 9], [10, 2, 3, 11], [12, 2, 3, 13], [14, 1, 15, 16], [17, 18, 19, 20]]
각 단어의 인덱스: {} {'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}
라벨:  [[1], [0], [1], [1], [0], [1]]


In [0]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label)) # Sequenced와 label을 행 단위로 묶어줌(input target)
iterator = dataset.make_one_shot_iterator()  # one_shot_iterator를 없애고 이따 해봐야지;
#make_one_shot_iterator는 one-hot 인코딩을 해주는게 아니라 위에서 묶인 데이터를 하나씩 쓰도록 만들어줌
next_data = iterator.get_next() # 이터레이터가 만들어준 순서형 데이터의 다음 데이터를 나오게 함

In [10]:
with tf.Session() as sess: # eager 모드는 한번 적고 난 후에 적용해볼것
  while True: #조건 없이 while 트루라니.. 
    try:
      print(sess.run(next_data)) # 실제 쓰이는 data는 next_data(iterator의 get_next가 적용된)
    except tf.errors.OutOfRangeError: #안하면 무한히 가더라. 왜 for 이나 그런게 아니라 while문을? len이 안통하나? 안통하네..;
        break

(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))


In [14]:
BATCH_SIZE = 2 # 배치를 두 개로 설정

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.batch(BATCH_SIZE) # 여기서 배치 크기 적용
iterator = dataset.make_one_shot_iterator() # 그리고 이터레이터가 2개씩 반복되도록 설정
next_data = iterator.get_next() # 다음 요소(배치) 수령?

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

(array([[4, 1, 5, 6],
       [7, 1, 8, 9]], dtype=int32), array([[1],
       [0]], dtype=int32))
(array([[10,  2,  3, 11],
       [12,  2,  3, 13]], dtype=int32), array([[1],
       [1]], dtype=int32))
(array([[14,  1, 15, 16],
       [17, 18, 19, 20]], dtype=int32), array([[0],
       [1]], dtype=int32))


In [15]:
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.shuffle(len(sequences)) # 얘가 섞어줌
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess: 
  while True:
    try:
      print(sess.run(next_data)) 
    except tf.errors.OutOfRangeError:
        break

(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))


In [16]:
EPOCH = 2

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.repeat(EPOCH) #얘가 2배로 만들어준건가? 셔플을 어디다 두는가가 관건인거 같고..
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess: 
  while True:
    try:
      print(sess.run(next_data)) 
    except tf.errors.OutOfRangeError:
        break

(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))
(array([4, 1, 5, 6], dtype=int32), array([1], dtype=int32))
(array([7, 1, 8, 9], dtype=int32), array([0], dtype=int32))
(array([10,  2,  3, 11], dtype=int32), array([1], dtype=int32))
(array([12,  2,  3, 13], dtype=int32), array([1], dtype=int32))
(array([14,  1, 15, 16], dtype=int32), array([0], dtype=int32))
(array([17, 18, 19, 20], dtype=int32), array([1], dtype=int32))


In [17]:
def mapping_fn(X, Y=None): # 다른 곳에서는 lambda 함수로 대체한 케이스
  input = {'x': X}
  label = Y
  return input, label

# 여러 input이 나눠진 데이터로 있을 수 있음. 위의 매핑 기능을 이용하여 그런걸 다 통합할 수 있음
# 이 것에 대해 좀 더 볼 수 있어야 겠음.. 
# 역시 위의 것은 X가 하나인 경우를 위해 만들어진 함수고, 다수 함수를 위해선 다르게 정의해야 함
#def mapping_fn(X1, X2, ..., Y=None):
#  input = {'x1': X1, 'x2' : } 이걸 손으로 해야하다니... 하지만 난 쪼렙이니 일단 하던거나 하자 ㅇㅇ
#  label = Y
#  return input, label
 
dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn)
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess: 
  while True:
    try:
      print(sess.run(next_data)) 
    except tf.errors.OutOfRangeError:
        break

({'x': array([4, 1, 5, 6], dtype=int32)}, array([1], dtype=int32))
({'x': array([7, 1, 8, 9], dtype=int32)}, array([0], dtype=int32))
({'x': array([10,  2,  3, 11], dtype=int32)}, array([1], dtype=int32))
({'x': array([12,  2,  3, 13], dtype=int32)}, array([1], dtype=int32))
({'x': array([14,  1, 15, 16], dtype=int32)}, array([0], dtype=int32))
({'x': array([17, 18, 19, 20], dtype=int32)}, array([1], dtype=int32))


In [20]:
BATCH_SIZE = 2
EPOCH = 2

def mapping_fn(X, Y=None):
  input = {'x': X}
  label = Y
  return input, label

dataset = tf.data.Dataset.from_tensor_slices((sequences, label))
dataset = dataset.map(mapping_fn) # 매핑이 먼저되고
dataset = dataset.shuffle(len(sequences)) # 셔플한 다음에
dataset = dataset.batch(BATCH_SIZE) # 배치를 정해주고
dataset = dataset.repeat(EPOCH) # 반복을 만들었다. 
iterator = dataset.make_one_shot_iterator()
next_data = iterator.get_next()

with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_data))
    except tf.errors.OutOfRangeError:
      break

({'x': array([[ 7,  1,  8,  9],
       [10,  2,  3, 11]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
({'x': array([[14,  1, 15, 16],
       [17, 18, 19, 20]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
({'x': array([[ 4,  1,  5,  6],
       [12,  2,  3, 13]], dtype=int32)}, array([[1],
       [1]], dtype=int32))
({'x': array([[10,  2,  3, 11],
       [17, 18, 19, 20]], dtype=int32)}, array([[1],
       [1]], dtype=int32))
({'x': array([[14,  1, 15, 16],
       [ 4,  1,  5,  6]], dtype=int32)}, array([[0],
       [1]], dtype=int32))
({'x': array([[12,  2,  3, 13],
       [ 7,  1,  8,  9]], dtype=int32)}, array([[1],
       [0]], dtype=int32))


In [0]:
# 마지막 셀이 보통 사용되는 기능들을 모두 넣은 데이터이다. 
# tf.data.Dataset.from_tensor_slices(x1, ... , y) 로 텐서플로에 넣기 좋게 여러 데이터 셋을 동시에 메모리에 싣는다. 
# 위의 객체로 생성된 것을 data라고 할 때, data에는 매핑(.map()), 섞기(.shuffle(길이)), 쪼개기(.batch(배치크기)), 배수복제(.repeat(횟수))를 적용한다.
# 셔플이나 배치 쪼개기, epoch 등은 순서 한번 고려해봐도 될려나
# tf.data는 len이 없었다. 그래서 while문을 썼었다. 추후에 같은 프로세스를 eager 모드 등으로 한번 해볼 필요가 있음